📊 Comprehensive Report: Grocery Inventory Analysis¶
# 🛒 Grocery Inventory Analysis

This data analysis project focused on grocery inventory.  
It includes data cleaning, aggregation, and visualization to derive meaningful insights on product categories, prices, and shelf life.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import the dataset
This heading signals the next step in the workflow: loading the dataset into a pandas DataFrame for analysis.

In [ ]:
grocery = pd.read_csv("/kaggle/input/grocery-inventory/Grocery_Inventory new v1.csv")

# See the result
grocery

# Missing values
This section addresses any missing or incorrect values in the dataset. Identifying and correcting these issues is essential for accurate analysis.

In [ ]:
grocery.isnull().sum()
grocery.loc[grocery['Product_Name'] == 'Cabbage', 'Catagory'] = 'Fruits & Vegetables'

This command provides a summary of the dataset, including column names, data types, and the number of non-null values per column. It's useful for understanding the structure of the data.

In [ ]:
grocery.info()

# Date Parsing and Year Adjustment
This section focuses on cleaning and converting date fields to ensure consistent formatting and proper analysis of time-based data.

In [ ]:
grocery['Date_Received'] = pd.to_datetime(grocery['Date_Received'], dayfirst=True, errors='coerce')
grocery['Last_Order_Date'] = pd.to_datetime(grocery['Last_Order_Date'], dayfirst=True, errors='coerce')
grocery['Expiration_Date'] = pd.to_datetime(grocery['Expiration_Date'], dayfirst=True, errors='coerce')

date_columns = ['Date_Received', 'Last_Order_Date', 'Expiration_Date']

def safe_replace_year_adjust(dt, new_year=2025):
    if pd.isna(dt):
        return pd.NaT
    try:
        return dt.replace(year=new_year)
    except ValueError:
        # Change only for the case of February 29
        if dt.month == 2 and dt.day == 29:
            return dt.replace(year=new_year, day=28)
        return pd.NaT
        
for col in date_columns:
    print(f"{col} years: {grocery[col].dt.year.unique()}")

#Running the replacement again only on the rows that are still 2024
for col in date_columns:
    grocery.loc[
        grocery[col].dt.year == 2024, col
    ] = grocery.loc[
        grocery[col].dt.year == 2024, col
    ].apply(lambda dt: safe_replace_year_adjust(dt, 2025))


# Expiration date check
This section calculates how long items remain in storage and flags items that are near expiration or already expired.

In [ ]:
grocery['Storage_Duration_Days'] = (grocery['Expiration_Date'] - grocery['Date_Received']).dt.days
grocery['Storage_Alert'] = grocery['Storage_Duration_Days'].apply(
    lambda x: 'Less than 30 days' if pd.notnull(x) and x < 30 else '')
today = pd.Timestamp.today()
grocery['Storage_Status'] = grocery['Expiration_Date'].apply(
    lambda x: 'Expired' if pd.notnull(x) and x < today else 'OK')

# Recalculate storage time only in rows where the time is negative or missing, taking into account-NaN
mask = grocery['Storage_Duration_Days'].notna() & (grocery['Storage_Duration_Days'] < 0) | grocery['Storage_Duration_Days'].isna()

# Storage time correction
grocery.loc[mask, 'Storage_Duration_Days'] = (
    (grocery.loc[mask, 'Expiration_Date'] - grocery.loc[mask, 'Date_Received']).dt.days
)
# Filtering invalid rows
grocery_cleaned = grocery[grocery['Storage_Duration_Days'].notna() & (grocery['Storage_Duration_Days'] >= 0)]

grocery_cleaned

# Price Cleaning
This part handles formatting and cleaning of the unit price data to ensure it can be analyzed as a numerical field.

In [ ]:
grocery_cleaned['Unit_Price'] = grocery_cleaned['Unit_Price'].str.replace('$', '', regex=False).astype(float)
grocery_cleaned = grocery_cleaned.rename(columns={'Unit_Price': 'Unit_Price_in_$'})
grocery_cleaned

# General Descriptive Statistics
Introduces the next phase of the analysis: computing summary statistics like averages and distributions for various attributes.

In [ ]:
desc_stats = grocery_cleaned.describe()

# Categories distribution
This section analyzes the distribution of products across different categories to identify which ones dominate the inventory.

In [ ]:
category_counts = grocery_cleaned['Catagory'].value_counts()

# Categories distribution graph
plt.figure(figsize=(10, 5))
sns.countplot(data=grocery_cleaned, y='Catagory', order=category_counts.index, palette='viridis')
plt.title('Product distribution by category')
plt.xlabel('Number of products')
plt.ylabel('Catagory')
plt.tight_layout()
plt.show()

# Average price by category
This section examines the average unit price for each product category to understand pricing trends across the inventory

In [ ]:
avg_price_per_category = grocery_cleaned.groupby('Catagory')['Unit_Price_in_$'].mean().sort_values()

# Average price per unit graph by category
plt.figure(figsize=(10, 5))
avg_price_per_category.plot(kind='barh', color='teal')
plt.title('Average price per unit by category')
plt.xlabel('Average price ($)')
plt.ylabel('Catagory')
plt.tight_layout()
plt.show()

# Correlation Between Stock and Sales
Introduces the next stage of analysis: exploring how inventory levels relate to sales metrics, potentially uncovering trends or inefficiencies

In [ ]:
correlation_stock_sales = grocery_cleaned[['Stock_Quantity', 'Sales_Volume']].corr()

In [ ]:
#Print Summary Stats and Correlation
print("Descriptive statistics:\n", desc_stats)
print("\nCorrelation between inventory quantity and sales:\n", correlation_stock_sales)

# Number of products in each category
Introduces an interactive bar chart to visualize product count by category.

In [ ]:
# Counting the number of products in each category.
category_counts = grocery_cleaned['Catagory'].value_counts().reset_index()
category_counts.columns = ['Category', 'Product_Count']

# Create an interactive bar graph
fig = px.bar(category_counts,
             x='Category',
             y='Product_Count',
             title='Interactive Comparison of Product Counts by Category',
             labels={'Product_Count': 'Number of Products', 'Category': 'Category'},
             hover_data=['Product_Count'])

fig.update_layout(xaxis_tickangle=-45)
fig.show()

# Average storage time by category
Creates an interactive bar chart to visualize average shelf life by category, which can inform stock management decisions.

In [ ]:
avg_storage_duration_cleaned = grocery_cleaned.groupby('Catagory')['Storage_Duration_Days'].mean().reset_index()
avg_storage_duration_cleaned.columns = ['Category', 'Average_Storage_Duration_Days']

# Interactive graph
fig = px.bar(avg_storage_duration_cleaned,
             x='Category',
             y='Average_Storage_Duration_Days',
             title='Average Storage Duration by Category (Cleaned Data)',
             labels={'Average_Storage_Duration_Days': 'Average Storage Duration (Days)', 'Category': 'Category'},
             hover_data=['Average_Storage_Duration_Days'])

fig.update_layout(xaxis_tickangle=-45)
fig.show()

# Calculating average price by category
Investigates pricing trends by showing average unit prices in each product category

In [ ]:
avg_price_per_category = grocery_cleaned.groupby('Catagory')['Unit_Price_in_$'].mean().reset_index()
avg_price_per_category.columns = ['Category', 'Average_Unit_Price']

# Interactive graph
fig = px.bar(avg_price_per_category,
             x='Category',
             y='Average_Unit_Price',
             title='Average Unit Price by Category',
             labels={'Average_Unit_Price': 'Average Price ($)', 'Category': 'Category'},
             hover_data=['Average_Unit_Price'])

fig.update_layout(xaxis_tickangle=-45)
fig.show()

# Total Inventory Cost per Category
Summarizes the overall financial value of stock in each category.

In [ ]:
grocery_cleaned['Total_Value'] = grocery_cleaned['Unit_Price_in_$'] * grocery_cleaned['Stock_Quantity']
value_per_category = grocery_cleaned.groupby('Catagory')['Total_Value'].sum().reset_index()

# Pie Chart of Inventory Value
fig = px.pie(value_per_category,
             names='Catagory',
             values='Total_Value',
             title='Total Inventory Value by Category',
             hole=0.4)
fig.show()


# The relationship between the cost of a product and its shelf life
Explores the relationship between a product’s price and how long it can be stored.

In [ ]:
fig = px.scatter(grocery_cleaned,
                 x='Storage_Duration_Days',
                 y='Unit_Price_in_$',
                 color='Catagory',
                 size='Stock_Quantity',
                 title='Unit Price vs. Storage Duration by Category',
                 hover_data=['Product_Name'])
fig.show()


# Shows the dispersion, median, and outliers of product prices in each category
Shows price distributions and highlights outliers using an interactive boxplot, helping identify pricing inconsistencies

In [ ]:
fig = px.box(grocery_cleaned,
             x='Catagory',
             y='Unit_Price_in_$',
             title='Price Distribution by Category',
             points='all')  # Also shows unusual points
fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Save to a new file

In [ ]:
grocery_cleaned.to_csv("Grocery_Inventory_data.csv", index=False)

# Final Report and Insights

Data Cleaning:
    Converted Date_Received and Expiration_Date columns to datetime.
    Fixed negative or missing Storage_Duration_Days using date differences.
    Removed rows with still-missing or negative values after correction.

Feature Engineering:
    Calculated storage duration (Storage_Duration_Days).
    Calculated total value (Unit_Price_in_$ × Stock_Quantity).

Aggregation & Grouping:
    Average storage duration by category.
    Average unit price by category.
    Total inventory value by category.

Visualization:
    Bar charts of average storage duration and unit price.
    Pie chart of total inventory value.
    Scatter plot of unit price vs. storage duration by category.

🔍 Key Findings 
    Most stocked category: Fruits & Vegetables
    Longest shelf life: Oils & Fats
    Shortest shelf life: Seafood
    Most expensive: Beverages

✅ Recommendations

Control Cost and Stock:
    Monitor expensive categories like Beverages for cost-efficiency.
    Combine pricing with sales volume to identify imbalances.

Optimize Shelf Life:
    High average duration may suggest durability or slow turnover.
    Categories like Seafood require frequent stock refresh due to shorter shelf life.

Expand Analysis:
    Include supplier-level insights for better vendor management.
    Correlate sales and inventory data to identify sales opportunities.

Report generated with Python and Plotly from a cleaned and validated grocery inventory dataset.